**извлечение признаков из текста на естественном языке**

Евгений Борисов borisov.e@solarl.ru

In [2]:
# http://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html

# разложение частотной матрицы [ слова x документы ]  
#
# получаем матрицу с описанием тем [ слова х темы ]   
# и матрицу вероятностей событий "тема описывает документ"  [ темы х документы ]
# 
# [ слова x документы ] = [ слова х темы ] * [ темы х документы ]
# 
# p(w|d) = p(w|t) * p(t|d)

import sys
import re
import gzip
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF

In [3]:
pd.options.display.max_colwidth = 200  

In [4]:
n_features = 1000
n_components = 10

df = pd.read_pickle('../data/text/news.pkl.gz')
print('текстов:',len(df))
df.sample(4)

текстов: 3196


,text,tag
535,В Ляховичском районе 16-летний рыбак провалился под лед\n\n6 декабря 2016 в 12:27\n\nTUT.BY\n\n3 декабря на водохранилище Нетчин в деревне Кривошин 11-классник во время рыбалки провалился под лед....,incident
806,"""Вежливые люди"" 1939 года. История глазами стариков, переживших шесть властей\n\n5 декабря 2016 в 14:52\n\nTUT.BY\n\nЖурналист и писатель Василий Сарычев уже пятнадцать лет записывает воспоминания...",politics
3072,"События Ноябрьска. ""Газпром нефть"" поддержит социальные инициативы\nгородов и поселков Ямала Компания ""Газпром нефть"" поддержит социальные\nинициативы городов и поселков Ямала и Югры. Таким образ...",economics
1762,"Ольга Полякова может стать зампредом Центробанка – «Коммерсантъ»\n\nНовый член совета директоров Банка России, руководитель главного управления по Центральному федеральному округу Ольга Полякова м...",economics


In [5]:
data = df['text'].tolist()

In [38]:
with gzip.open('../data/text/stop-nltk.txt.gz','rt',encoding='utf-8') as f: 
    stopwords = set([ w.strip() for w in  f.read().split() if w.strip() ] )

print('количество стоп-слов:',len(stopwords))

sorted(stopwords)

количество стоп-слов: 151


In [40]:
# # очистка текста 
# def preprocessor(tt):
#     tt = [ t.lower() for t in tt ] # приведение в lowercase
#     tt = [ re.sub( r'\W', ' ', t)  for t in tt ] # удаление лишних символов (НЕ буква и НЕ цифра)
#     tt = [ re.sub( r'_', ' ', t)  for t in tt ] 
#     tt = [ re.sub( r'\b\d+\b', ' ', t ) for t in tt ] # замена цифр
#     tt = [ re.sub( r'\b.*\d+.*\b', ' ', t ) for t in tt ]  # замена буквенно-цифровых кодов
#     tt = [ re.sub( r'[a-z]+', ' ', t ) for t in tt ]  # удаление слов из символов латиницы
#     return [ t.strip() for t in tt if len(t.strip())>2 ] # удаление коротких слов

In [41]:
%%time 

df['text_clean'] = df['text'].str.lower() # приведение в lowercase

# замена символов-разделителей (-,_) на пробел
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'\W', ' ', s))
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'_', ' ', s))

# замена цифр
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'\b\d+\b', ' ', s))

# делим строки на слова
df['text_clean'] = df['text_clean'].apply(lambda t: [ w.strip() for w in t.split() if len(w.strip())>2 ] )

# удаление лишних слов
df['text_clean'] = df['text_clean'].apply(lambda t:[w for w in t if w not in stopwords])

CPU times: user 588 ms, sys: 6.65 ms, total: 594 ms
Wall time: 595 ms


In [42]:
df[['text_clean']].sample(10)

,text_clean
2893,"[данном, видео, которая, выложила, kia, новая, модель, набирает, сотню, секунды, дает, производителю, основания, называть, грядущую, новинку, самой, динамичной, моделью, компании, первом, видео, к..."
2963,"[эксперты, автомобильной, отрасли, провели, масштабное, исследование, отечественного, авторынка, целью, определить, какие, марки, модели, автомобилей, акпп, россии, продавались, других, текущем, г..."
3166,"[уралинформбюро, uralinform, екатеринбург, тундровикам, надымского, района, доставили, паспорта, вертолете, представителей, коренных, малочисленных, народов, севера, живущих, удаленном, ямальском,..."
3100,"[regnum, хабаровске, пройдет, фестиваль, художественных, ремесел, коренных, народов, хабаровске, августа, рамках, второго, международного, десятилетия, коренных, народов, мира, пройдет, дальневост..."
271,"[walkman, обзавелся, кредлом, компания, brando, широко, известная, своими, аксессуарами, выпустила, кредл, который, предназначен, последних, моделей, телефонов, sony, ericsson, списке, поддерживае..."
1470,"[мнению, лидера, радикальной, партии, олега, ляшко, действия, нардепа, надежды, савченко, это, государственная, измена, которую, лишить, парламентария, мандата, народного, депутата, чудит, савченк..."
854,"[названы, худшие, фильмы, года, декабря, afisha, tut, издание, business, insider, опубликовало, антирейтинг, фильмов, вышедших, прокат, году, помощью, агрегатора, отзывов, metacritic, собраны, фил..."
275,"[состоялась, встреча, президентов, украины, польши, ноября, президенту, республики, польша, александру, квасьневскому, президент, украины, виктор, ющенко, вручил, орден, заслуги, степени, мариинск..."
2647,"[ученые, университета, калифорнии, сша, провели, обширное, исследование, установили, женщины, мужчины, выбирают, партнеров, составили, критерии, которыми, руководствуются, современные, люди, выбор..."
232,"[дмитрий, певцов, даст, перми, концерт, пользу, восстановления, православной, святыни, известный, артист, дмитрий, певцов, даст, перми, концерт, пользу, восстановления, православной, святыни, бело..."


In [43]:
# sorted(set([ w for t in  df['text_clean'] for w in t ]))

In [44]:
# собираем слова в строку
df['text_clean'] = df['text_clean'].apply(lambda t: ' '.join(t) )

In [45]:
%xdel stopwords

In [46]:
df[ df['text_clean'].str.len()<1 ]

,text,tag,text_clean


In [34]:
df.sample(10)

,text,tag,text_clean
837,С прежним председателем и фуршетом. Как прошел съезд официального союза писателей\n\n30 ноября 2016 в 0:10\n\nАнастасия Бойко / Фото: Дмитрий Брушко / TUT.BY\n\nСъезд официального Союза писателей ...,culture,прежним председателем фуршетом прошел съезд официального союза писателей ноября анастасия бойко фото дмитрий брушко съезд официального союза писателей прошел ноября белгосфилармонии сходил меропри...
1310,Оскорбленная невеста запустила флешмоб из-за дешевого обручального кольца\n\n6 декабря 2016 в 14:10\n\nLenta.ru\n\nАмериканка Ариэль Дезир Макрей (Ariel Desiree McRae) запустила флешмоб о настояще...,tech,оскорбленная невеста запустила флешмоб дешевого обручального кольца декабря американка ариэль дезир макрей запустила флешмоб настоящей любви сотрудники ювелирного магазина назвали выбранное женихо...
1725,Японский банк Sumitomo Mitsui станет сотрудничать с российской группой ВТБ\n\nБанковская групп ВТБ намерена подписать соглашение о сотрудничестве с японским банком Sumitomo Mitsui. Соответствующий...,economics,японский банк станет сотрудничать российской группой втб банковская групп втб намерена подписать соглашение сотрудничестве японским банком соответствующий документ подписан ближайшее время рамках ...
704,Скидки и дегустации. В Гродненской области стартуют предпраздничные ярмарки\n\n9 декабря 2016 в 12:51\n\nБЕЛТА\n\nПредпраздничные ярмарки в Гродненской области стартуют с акции «Мясныя прысмакi». ...,reclama,скидки дегустации гродненской области стартуют предпраздничные ярмарки декабря белта предпраздничные ярмарки гродненской области стартуют акции мясныя прысмак накануне нового года рождества гродне...
788,"Ученые предрекли скорое вымирание жирафов\n\n8 декабря 2016 в 13:12\n\nTUT.BY\n\nИсследователи из Международного союза охраны природы (IUCN) заявили о том, что жирафы находятся на грани вымирания,...",tech,ученые предрекли скорое вымирание жирафов декабря исследователи международного союза охраны природы заявили жирафы находятся грани вымирания сообщает фото пециалисты подсчитали популяцию этих живо...
2783,"Сообщается, что довольно большое количество пользователей устройств от всемирно известной компании Apple начали жаловаться на повышенное количество спама. Стоит отметить, что это не обычный спам, ...",tech,сообщается довольно большое количество пользователей устройств всемирно известной компании начали жаловаться повышенное количество спама стоит отметить это обычный спам которым компании привыкли б...
5,"Продюсер сериала ""Секс в большом городе"" Даррен Стар вместе с ""Би-Би-Си""\nработает над новым проектом, в этот раз о гонках ""Формула-1"". Новый\nсериал пока не имеет названия. Но уже известно, что ...",culture,продюсер сериала секс большом городе даррен стар вместе работает новым проектом гонках формула новый сериал пока имеет названия известно рассказывать изнанке формулы роскоши которая окружает гонщи...
1859,"Избранный президент США Дональд Трамп заявил, что бывший губернатор штата Массачусетс Митт Ромни не займет пост госсекретаря. Об этом сообщает агентство Bloomberg.\n\nчитайте также Газета Financia...",politics,избранный президент сша дональд трамп заявил бывший губернатор штата массачусетс митт ромни займет пост госсекретаря сообщает агентство читайте также газета назвала трампа человеком года назначен ...
739,"Смерть младенца в роддоме Новополоцка: в действиях медиков преступления нет, но врач уволилась\n\n8 декабря 2016 в 11:27\n\nTUT.BY\n\nУправление Следственного комитета по Витебской области заверши...",incident,смерть младенца роддоме новополоцка действиях медиков преступления врач уволилась декабря управление следственного комитета витебской области завершило проверку факту смерти новорожденного новопол...
1570,"Сирийские войска на 98% освободили Алеппо\n\nПравительственные войска Сирии захватили 98% Алеппо, заявили в российском Центре примирения враждующих сторон. Под контролем боевиков осталось не больш...",politics,сирийские войска освободили алеппо правител

---

In [47]:
def print_top_words(model, feature_names, n_top_words=7):
    for topic_idx, topic in enumerate(model.components_):
        message = "Тема %d: " % topic_idx
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)

---

In [48]:
# tf features 
tf_vectorizer = CountVectorizer( max_df=0.95, min_df=2, max_features=n_features )
tf = tf_vectorizer.fit_transform(df['text_clean'])
tf_feature_names = tf_vectorizer.get_feature_names()

In [49]:
# LDA - латентное размещение Дирихле
lda = LatentDirichletAllocation( n_components=n_components, max_iter=5, 
                                learning_method='online', learning_offset=50.,
                                random_state=0 ).fit(tf)
print('\nLDA:\n')
print_top_words(lda, tf_feature_names)


LDA:

Тема 0: это очень которые время просто лет нужно
Тема 1: президент сша трамп президента заявил декабря глава
Тема 2: рублей автомобиль модели автомобилей модель автомобиля авто
Тема 3: беларуси tut декабря установлена внимание версия браузер
Тема 4: народов коренных севера малочисленных коми проекта края
Тема 5: object савченко нефти нефть тгк индекс оао
Тема 6: года году это также компании декабря которые
Тема 7: декабря tut время дтп фото результате области
Тема 8: года также декабря россии тысяч сообщает данным
Тема 9: динамо лучший мяч мира место стал чемпионата


---

In [50]:
# tf-idf features 
tfidf_vectorizer = TfidfVectorizer( max_df=0.95, min_df=2, max_features=n_features)

tfidf = tfidf_vectorizer.fit_transform(df['text_clean'])
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [51]:
# NMF (Frobenius norm) - неотрицательное матричное разложение
nmf = NMF( n_components=n_components, random_state=1,alpha=.1, l1_ratio=.5 ).fit(tfidf)
print('\nNMF(Frobenius norm):\n')
print_top_words( nmf, tfidf_feature_names )


NMF(Frobenius norm):

Тема 0: это года которые году также декабря время
Тема 1: трамп сша трампа дональд президент избранный президента
Тема 2: дтп водитель результате области мвд декабря происшествия
Тема 3: flash adobe javascript player проигрывателя html5 браузер
Тема 4: савченко украины партии надежда заявила лидер действия
Тема 5: рублей млн млрд долларов тысяч года году
Тема 6: динамо чемпионата матче мира очков матча лиги
Тема 7: народов севера коренных малочисленных края фестиваль июля
Тема 8: россии путин президент президента заявил глава россия
Тема 9: алеппо города сирии жителей восточной сутки тысяч


---

In [52]:
# NMF (generalized Kullback-Leibler divergence)  
nmf = NMF( n_components=n_components, random_state=1, beta_loss='kullback-leibler', 
          solver='mu', max_iter=1000, alpha=.1, l1_ratio=.5 ).fit(tfidf)
print('\nNMF(generalized Kullback-Leibler divergence):\n')
print_top_words(nmf, tfidf_feature_names )


NMF(generalized Kullback-Leibler divergence):

Тема 0: это фото также года декабря которая который
Тема 1: сша президент трамп президента ранее трампа также
Тема 2: декабря результате области человек сообщает сообщили пресс
Тема 3: декабря tut установлена поддерживает версия фото старая
Тема 4: это сегодня украины словам лидер декабря является
Тема 5: рублей также млн рамках области пресс народов
Тема 6: стал мира место декабря который чемпионата года
Тема 7: ученые человека которые людей человек лет жизни
Тема 8: россии декабря заявил страны президент также словам
Тема 9: также сообщает ранее года компания который тысяч
